In [1]:
import torch
from torch.utils.data import Dataset
from datasets import load_dataset
from datasets import load_from_disk
from transformers import AlbertForSequenceClassification, AlbertTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding

In [2]:
# Loading the tokenized datasets
token_train = load_from_disk('/Users/negarakhgar/Desktop/nlp project/tokenized_test')
token_test = load_from_disk('/Users/negarakhgar/Desktop/nlp project/tokenized_train')

In [3]:
# Loading ALBERT model for binary classification
model = AlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=2)

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
pip install -U accelerate transformers torch

  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.7 kB)
Using cached transformers-4.49.0-py3-none-any.whl (10.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 MB 2.6 MB/s eta 0:00:0000:0100:01
Using cached tokenizers-0.21.0-cp39-abi3-macosx_11_0_arm64.whl (2.6 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1
    Uninstalling torch-2.5.1:
      Successfully uninstalled torch-2.5.1
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may ne

In [6]:
# Setting training arguments
training_args = TrainingArguments(
    output_dir='/Users/negarakhgar/Desktop/nlp project',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01
)

In [7]:
# Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=token_train,
    eval_dataset=token_test
)

In [8]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.304300,0.300965
2,0.218700,0.334869
3,0.136200,0.484847


TrainOutput(global_step=4689, training_loss=0.23216699765775473, metrics={'train_runtime': 10228.5579, 'train_samples_per_second': 7.332, 'train_steps_per_second': 0.458, 'total_flos': 448089408000000.0, 'train_loss': 0.23216699765775473, 'epoch': 3.0})

In [9]:
# Evaluate the model on the test set
results = trainer.evaluate()

print(results)

{'eval_loss': 0.48484671115875244, 'eval_runtime': 779.501, 'eval_samples_per_second': 32.072, 'eval_steps_per_second': 0.502, 'epoch': 3.0}


In [10]:
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

In [11]:
# Saving the fine-tuned model and tokenizer
model.save_pretrained('/Users/negarakhgar/Desktop/nlp project/model')
tokenizer.save_pretrained('/Users/negarakhgar/Desktop/nlp project/model')

('/Users/negarakhgar/Desktop/nlp project/model/tokenizer_config.json',
 '/Users/negarakhgar/Desktop/nlp project/model/special_tokens_map.json',
 '/Users/negarakhgar/Desktop/nlp project/model/spiece.model',
 '/Users/negarakhgar/Desktop/nlp project/model/added_tokens.json')

In [26]:
model1 = AlbertForSequenceClassification.from_pretrained('/Users/negarakhgar/Desktop/nlp project/model')
tokenizer1 = AlbertTokenizer.from_pretrained('/Users/negarakhgar/Desktop/nlp project/model')

In [29]:
model1.eval()

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertSdpaAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=

In [13]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)

In [ ]:
tokenized_test_data = token_test.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [18]:
pip install evaluate


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
import evaluate

accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

In [21]:
from transformers import EvalPrediction

In [22]:
def compute_metrics(eval_pred: EvalPrediction):
    logits, labels = eval_pred
    logits = torch.tensor(logits)
    labels = torch.tensor(labels)

    predictions = torch.argmax(logits, dim=1)
    
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)['accuracy']
    precision = precision_metric.compute(predictions=predictions, references=labels, average='weighted')['precision']
    recall = recall_metric.compute(predictions=predictions, references=labels, average='weighted')['recall']
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')['f1']

    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}

In [23]:
eval_args = TrainingArguments(
    output_dir='/Users/negarakhgar/Desktop/nlp project/evaluation',
    per_device_eval_batch_size=32
)

In [27]:
trainer_albert = Trainer(
    model=model1,
    args=eval_args,
    eval_dataset=tokenized_test_data,
    compute_metrics=compute_metrics
)

In [31]:
# Evaluating the model
results_albert = trainer_albert.evaluate()
print(f"ALBERT Evaluation Results:\n{results_albert}")

ALBERT Evaluation Results:
{'eval_loss': 0.30201759934425354, 'eval_model_preparation_time': 0.0066, 'eval_accuracy': 0.93392, 'eval_precision': 0.9339200111083523, 'eval_recall': 0.93392, 'eval_f1': 0.933919999577088, 'eval_runtime': 216598.4724, 'eval_samples_per_second': 0.115, 'eval_steps_per_second': 0.004}
